In [1]:
from util._io import read_config
from collections import namedtuple
from pathlib import Path
import pandas as pd
import numpy as np
import fastsca
import logging

In [2]:
experiment = '/projects/experiments/fastsca/'
server = Path.home().as_posix()
experiment_home = server+experiment
experiment_config = read_config(experiment_home+'config.yaml')
args = namedtuple('Struct',experiment_config.keys())(*experiment_config.values())

sca = fastsca.FASTSCA()
sca.config = args
sca.initdata()
sca.loaddata()
sca.data.mtx.shape

(155913, 20265)

In [3]:
fpath = sca.config.home + sca.config.experiment +sca.config.output + sca.config.sample_id+'/'+sca.config.sample_id
df = pd.read_csv(fpath+"_rp_bulk.csv.gz")
df.shape


(1429, 20265)

##### Compare random projected data with the original data before modeling 

Idea from ->

Efficient Generation of Transcriptomic Profiles by Random Composite Measurements

Similarity of Expression Profiles: To test the method, we applied it to each dataset, comparing the pairwise distances in the original high-dimensional space X (full original dataset), with those between samples as represented in the low-dimensional embedding, Y (compressed dataset). With only m = 100 random composite measurements, the Pear- son correlation coefficient (averaged across the datasets) be- tween the distances in the low- and high-dimensional spaces was 94.4%. With m = 400 random measurements, the average correlation was 98.5%.

In this case,
$$Y_{N\times M} = X_{N \times G} R_{G \times M}$$

In [4]:
y = np.dot(sca.data.mtx,df.T.to_numpy())


In [14]:
from sklearn.metrics import DistanceMetric
dist = DistanceMetric.get_metric('euclidean')

y_dist = dist.pairwise(y)
x_dist = dist.pairwise(sca.data.mtx)
print(y_dist.shape, x_dist.shape)

y_dist_n = y_dist -y_dist.mean(1, keepdims=True)
x_dist_n = x_dist -x_dist.mean(1, keepdims=True)

# compute the row-wise Pearson correlation coefficients between the variables in x and y.
corr = []
for i in range(2700):
    corr.append(np.corrcoef(x_dist_n[i],y_dist_n[i])[0,1])
print(np.mean(corr))


(2700, 2700) (2700, 2700)
0.7922996271343656


# PBMC
- when depth is 4 and dimension is 10 then average correlation is 0.71
- when depth is 8 and dimension is 19 then average correlation is 0.68
- when depth is 16 and dimension is 173 then average correlation is 0.77
- when depth is 32 and dimension is 924 then average correlation is 0.79

# TNBC

Bulk dimension is (1429, 20265) - with min_leaf = 250, max_depth = 25

also,
Bulk dimension is (233, 20265) - with min_leaf = 1000, max_depth = 16
 - y = np.dot(sca.data.mtx,df.T.to_numpy()) 
 is taking >15 mins.
